In [2]:
import tensorflow_hub as hub
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
import cv2

In [4]:
model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')


In [6]:
def load_image(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = img[tf.newaxis, :]
    return img

content_image = load_image('img/turtle.jpg')
style_image = load_image('img/wave.jpg')


In [7]:
stylized_image = model(tf.constant(content_image), tf.constant(style_image))[0]


In [ ]:
plt.imshow(np.squeeze(content_image))
plt.title('Content Image')
plt.show()
plt.imshow(np.squeeze(style_image))
plt.title('Style Image')
plt.show()